In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
import re

In [54]:
train=pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/çalışma 2/new_train.csv")
test=pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/çalışma 2/new_test.csv")
submission=pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/çalışma 2/sample_submission.csv")

In [55]:
def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

train = encode(train, 'yil', 365)
train = encode(train, 'ay', 12)
train = encode(train, 'haftanin_gunu', 7)

test = encode(test, 'yil', 365)
test = encode(test, 'ay', 12)
test = encode(test, 'haftanin_gunu', 7)

In [56]:
silinecek_sutunlar=["yil","ay","haftanin_gunu","bildirimli_sum","tarih","mevsim","min_t_2m:C","max_t_2m:C",'mean_t_2m:C',
                    't_difference_2m:C','min_t_apparent:C','max_relative_humidity_2m:p','t_apparent_difference:C','max_global_rad:W',
                    'max_t_apparent:C','mean_t_apparent:C','tatil_mi']
train_df=train.drop(silinecek_sutunlar,axis=1)
test_df=test.drop(silinecek_sutunlar,axis=1)

In [57]:
train_df=train_df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
#train_df=pd.get_dummies(data=train_df, columns=["ilce"], dtype="int")
train_df.head()

,ilce,bildirimsiz_sum,max_wind_speed_10mms,max_prob_precip_1hp,yil_sin,yil_cos,ay_sin,ay_cos,haftanin_gunu_sin,haftanin_gunu_cos
0,izmir-aliaga,5.0,6.4,1.0,-0.230306,-0.973118,0.5,0.866025,-0.433884,-0.900969
1,izmir-kemalpasa,4.0,2.5,1.0,-0.230306,-0.973118,0.5,0.866025,-0.433884,-0.900969
2,izmir-guzelbahce,1.0,6.2,5.9,-0.230306,-0.973118,0.5,0.866025,-0.433884,-0.900969
3,izmir-bayrakli,7.0,3.5,3.7,-0.230306,-0.973118,0.5,0.866025,-0.433884,-0.900969
4,izmir-foca,8.0,7.2,12.8,-0.230306,-0.973118,0.5,0.866025,-0.433884,-0.900969


In [58]:
#test_df=pd.get_dummies(data=test_df, columns=["ilce"], dtype="int")
test_df = test_df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
test_df.head()

,ilce,max_wind_speed_10mms,max_prob_precip_1hp,yil_sin,yil_cos,ay_sin,ay_cos,haftanin_gunu_sin,haftanin_gunu_cos
0,izmir-aliaga,7.0,1.0,-0.280231,-0.959933,0.866025,0.5,0.433884,-0.900969
1,izmir-bayindir,2.8,1.0,-0.280231,-0.959933,0.866025,0.5,0.433884,-0.900969
2,izmir-bayrakli,3.3,1.0,-0.280231,-0.959933,0.866025,0.5,0.433884,-0.900969
3,izmir-bergama,5.8,1.0,-0.280231,-0.959933,0.866025,0.5,0.433884,-0.900969
4,izmir-bornova,3.6,1.0,-0.280231,-0.959933,0.866025,0.5,0.433884,-0.900969


In [59]:
y=train_df["bildirimsiz_sum"]
x=train_df.drop("bildirimsiz_sum",axis=1)

In [60]:
X_train=x[:int(len(x)*0.8)]
X_test=x[int(len(x)*0.8):]
y_train=y[:int(len(y)*0.8)]
y_test=y[int(len(y)*0.8):]
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(41322, 9) (10331, 9) (41322,) (10331,)


In [90]:
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate
params={"max_depth":[3,5,8,None,7,10],
            "learning_rate":[0.01,0.1],
            "iterations":[100,150,200,500,750,1000],
            "subsample":[0.6,0.8,0.9,1],
            "random_state":[14],
            #"cat_features":[0]
            }
from catboost import CatBoostRegressor
cat_model=CatBoostRegressor(cat_features=[0])
lgbm_best_grid=GridSearchCV(cat_model,
                           params,
                           cv=3,
                           n_jobs=-1,
                           verbose=True).fit(X_train, y_train)

Fitting 3 folds for each of 288 candidates, totalling 864 fits
0:	learn: 5.1719215	total: 7.34ms	remaining: 1.46s
1:	learn: 5.0445470	total: 12.5ms	remaining: 1.24s
2:	learn: 4.9365583	total: 17.3ms	remaining: 1.13s
3:	learn: 4.8429098	total: 22.1ms	remaining: 1.08s
4:	learn: 4.7649102	total: 27.1ms	remaining: 1.06s
5:	learn: 4.6958701	total: 31.9ms	remaining: 1.03s
6:	learn: 4.6378891	total: 36.7ms	remaining: 1.01s
7:	learn: 4.5905553	total: 41.6ms	remaining: 999ms
8:	learn: 4.5470675	total: 47.2ms	remaining: 1s
9:	learn: 4.5133401	total: 53.1ms	remaining: 1.01s
10:	learn: 4.4828842	total: 57.6ms	remaining: 990ms
11:	learn: 4.4564756	total: 63.3ms	remaining: 991ms
12:	learn: 4.4339843	total: 67.8ms	remaining: 976ms
13:	learn: 4.4152563	total: 72.7ms	remaining: 966ms
14:	learn: 4.3976886	total: 79.8ms	remaining: 985ms
15:	learn: 4.3827155	total: 86.1ms	remaining: 990ms
16:	learn: 4.3684322	total: 91.4ms	remaining: 984ms
17:	learn: 4.3565537	total: 96.4ms	remaining: 974ms
18:	learn: 4.3

In [91]:
best_params=lgbm_best_grid.best_params_
best_params

{'iterations': 200,
 'learning_rate': 0.1,
 'max_depth': 3,
 'random_state': 14,
 'subsample': 0.9}

In [92]:
from catboost import CatBoostRegressor
cat_model=CatBoostRegressor( **best_params, cat_features=[0], verbose=50)
cat_model.fit(X_train, y_train)
y_pred=cat_model.predict(X_test)
print("MAE: ",mean_absolute_error(y_test,y_pred))

0:	learn: 5.1719215	total: 6.85ms	remaining: 1.36s
50:	learn: 4.2100751	total: 278ms	remaining: 812ms
100:	learn: 4.1493897	total: 583ms	remaining: 572ms
150:	learn: 4.1220708	total: 887ms	remaining: 288ms
199:	learn: 4.1002427	total: 1.17s	remaining: 0us
MAE:  2.9720046898170094


In [93]:
cross_val_score(cat_model,
                X_train,
                y_train,
                scoring="neg_mean_squared_error",
                cv=10,
                n_jobs=-1)

array([-36.3975334 , -21.09550808, -17.40514267, -25.29707376,
       -19.6616021 ,  -9.7308848 , -12.19350497, -13.07065468,
       -12.47670306, -14.56972493])

In [100]:
#res=cat_model.calc_feature_statistics(X_train, y_train, feature=2, plot=True)

In [101]:
test["bildirimsiz_sum"] = cat_model.predict(test_df)
test["bildirimsiz_sum"]=test["bildirimsiz_sum"].astype("int")
test["unique_id"]=[str(test["tarih"][i])+"-"+str(test["ilce"][i]) for i in range(0,len(test))]
predict=pd.DataFrame()
predict=test[["unique_id","bildirimsiz_sum"]]
predict["bildirimsiz_sum"]=test["bildirimsiz_sum"]
submission1=pd.merge(submission,predict,how="left",on="unique_id")
submission1.drop("bildirimsiz_sum_x",axis=1,inplace=True)
submission1.columns=["unique_id","bildirimsiz_sum"]
print(submission1.isnull().sum())
submission1.head()

unique_id          0
bildirimsiz_sum    0
dtype: int64


<ipython-input-101-f9dc2e313d2d>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,unique_id,bildirimsiz_sum
0,2024-02-01-izmir-aliaga,4
1,2024-02-01-izmir-bayindir,3
2,2024-02-01-izmir-bayrakli,3
3,2024-02-01-izmir-bergama,5
4,2024-02-01-izmir-bornova,6


In [102]:
submission1.to_csv("/content/gdrive/My Drive/Colab Notebooks/çalışma 2/GDZ-Tahmin 14-CatBoost.csv",index=False)